In [1]:
import pandas as pd

In [2]:
f"{pd.to_datetime('today'):%Y-%m-%d}"

'2021-06-16'

# New information

[Minimally Sufficient Pandas](https://medium.com/dunder-data/minimally-sufficient-pandas-a8e67f2a2428) - Excellent!

- Biggest things I learned: 
    - Selecting a single column of data &rarr; use brackets
    - loc and iloc
    - isna vs isnull and notna vs notnull
    - Standardizing groupby aggregation
    - Handling a MultiIndex


# Information

[So you want to become a Data Scientist?](https://github.com/FBosler/you-datascientist)  
[Pandas GroupBy: Your Guide to Grouping Data in Python](https://realpython.com/pandas-groupby/)  

## [WHEN TO USE AGGREAGATE/FILTER/TRANSFORM WITH PANDAS](https://pythonforbiologists.com/when-to-use-aggregatefiltertransform-in-pandas/)

>I think that most of the confusion arises because the same grouping logic is used for (at least) three distinct operations in Pandas. In the order that we normally learn them, these are:

>- calculating some aggregate measurement for each group (size, mean, etc.)
- filtering the rows on a property of the group they belong to
- calculating a new value for each row based on a property of the group.

>This leads commonly to situations where we know that we need to use groupby() - and may even be able to easily figure out what the arguments to groupby() should be - but are unsure about what to do next.

>Here's a trick that I've found useful when teaching these ideas: think about the result you want, and work back from there. If you want to get a single value for each group, use aggregate() (or one of its shortcuts). If you want to get a subset of the original rows, use filter(). And if you want to get a new value for each original row, use transpose().

To reiterate:

>- if we want to get a single value for each group &rarr; use aggregate()  
>- if we want to get a subset of the input rows &rarr; use filter()  
>- if we want to get a new value for each input row &rarr; use transform()

### Load example data

In [ ]:
df = pd.read_csv(
   "https://raw.githubusercontent.com/mwaskom/seaborn-data/master/tips.csv"
   )

### Set max rows to display

In [2]:
pd.options.display.max_rows = 10
df

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2


### Average bill on each day

In [3]:
df.groupby('day').aggregate('mean')

,total_bill,tip,size
day,,,
Fri,17.151579,2.734737,2.105263
Sat,20.441379,2.993103,2.517241
Sun,21.410000,3.255132,2.842105
Thur,17.682742,2.771452,2.451613


In [4]:
df.groupby('day')['total_bill'].aggregate('mean')

day
Fri     17.151579
Sat     20.441379
Sun     21.410000
Thur    17.682742
Name: total_bill, dtype: float64

This means the same thing, i.e., is a shortcut for `aggregate('mean')`:

In [5]:
df.groupby('day')['total_bill'].mean()

day
Fri     17.151579
Sat     20.441379
Sun     21.410000
Thur    17.682742
Name: total_bill, dtype: float64

### Which meals eat on days when ave bill > \$20

>For this question, think again about the output we want - our goal here is to get a subset of the original rows, so this is a job for `filter()`. The argument to `filter()` must be a function or lambda that will take a group and return `True` or `False` to determine whether rows belonging to that group should be included in the output.

In [6]:
df.groupby('day').filter(
    lambda x : x['total_bill'].mean() > 20
    )

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
...,...,...,...,...,...,...,...
238,35.83,4.67,Female,No,Sat,Dinner,3
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2


>Compared to our first example, it's a bit harder to see why this is useful - typically we'll do a filter like this and then follow it up with another operation. For example, we might want to compare the average party size on days where the average bill is high:

In [7]:
df.groupby('day').filter(
    lambda x : x['total_bill'].mean() > 20
)['size'].mean()

2.668711656441718

>with the average party size on days where the average bill is low:

In [8]:
df.groupby('day').filter(
    lambda x : x['total_bill'].mean() <= 20
)['size'].mean()

2.3703703703703702

### Normalize cost of each meal for average for the day

In [11]:
df['scaled bill'] = df.groupby('day')['total_bill'].transform(
    lambda x : x/x.mean()
)
df.head()

,total_bill,tip,sex,smoker,day,time,size,scaled bill
0,16.99,1.01,Female,No,Sun,Dinner,2,0.793554
1,10.34,1.66,Male,No,Sun,Dinner,3,0.482952
2,21.01,3.50,Male,No,Sun,Dinner,3,0.981317
3,23.68,3.31,Male,No,Sun,Dinner,2,1.106025
4,24.59,3.61,Female,No,Sun,Dinner,4,1.148529
